In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset, Dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from tqdm import tqdm

In [3]:
#Reading csv file
train = pd.read_csv("/content/drive/MyDrive/DL_DialogDataset/train.csv")
test = pd.read_csv("/content/drive/MyDrive/DL_DialogDataset/test.csv")
train

,utterance,act
0,"Say , Jim , how about going for a few beers af...",3
1,You know that is tempting but is really not g...,4
2,What do you mean ? It will help us to relax .,2
3,Do you really think so ? I don't . It will ju...,2
4,I guess you are right.But what shall we do ? ...,2
...,...,...
5085,Tom and Helen got married at last .,1
5086,How did you know that ? I heart Tom ’ s fathe...,2
5087,I was invited to attend their wedding .,1
5088,It ’ s great . Although his father didn ’ t a...,1


In [4]:
Xtrain = train.utterance
ytrain = train.act
Xtest = test.utterance
ytest = test.act
  

## Preprocessing

In [5]:
!pip install contractions 

     |████████████████████████████████| 284 kB 5.3 MB/s 
     |████████████████████████████████| 106 kB 56.1 MB/s 


In [6]:
def lowerCase(df):
  df= df.apply(lambda x: x.lower())
  return df

Xtrain = lowerCase(Xtrain)
Xtest = lowerCase(Xtest)

In [7]:
import contractions
def contractionExpand(text):
  return contractions.fix(text)

Xtrain = Xtrain.apply(lambda x: contractionExpand(x))

In [8]:
Xtest = Xtest.apply(lambda x: contractionExpand(x))

In [9]:
import nltk
nltk.download('punkt')

def tokenize(text):
  tokens = nltk.word_tokenize(text)
  return tokens

Xtrain = Xtrain.apply(lambda x: tokenize(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
Xtest = Xtest.apply(lambda x: tokenize(x))

In [11]:
Xtrain

0       [say, ,, jim, ,, how, about, going, for, a, fe...
1       [you, know, that, is, tempting, but, is, reall...
2       [what, do, you, mean, ?, it, will, help, us, t...
3       [do, you, really, think, so, ?, i, do, not, .,...
4       [i, guess, you, are, right.but, what, shall, w...
                              ...                        
5085         [tom, and, helen, got, married, at, last, .]
5086    [how, did, you, know, that, ?, i, heart, tom, ...
5087     [i, was, invited, to, attend, their, wedding, .]
5088    [it, ’, s, great, ., although, his, father, di...
5089                           [how, moving, love, is, .]
Name: utterance, Length: 5090, dtype: object

In [12]:
Xtest

0      [i, am, better, now, ., want, to, play, again, ?]
1      [i, will, let, you, break, the, balls, this, t...
2      [let, us, get, all, the, balls, out, of, the, ...
3      [ok, ., how, much, do, you, want, to, bet, on,...
4      [you, are, crazy, ., gambling, is, against, my...
                             ...                        
717    [yeah, ,, i, got, to, eat, as, much, pizza, as...
718                       [did, you, like, the, play, ?]
719    [not, really, ., it, is, a, dull, one, ,, and,...
720    [you, are, absolutely, right, ., the, acting, ...
721    [to, be, fair, ,, though, ,, both, the, costum...
Name: utterance, Length: 722, dtype: object

In [13]:
max_seq_len = len(max(Xtrain, key=len))

In [14]:
max_seq_len_test = len(max(Xtest, key=len))

### Sentence Embedding for each utterance: Using Glove and then pooling

In [15]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!ls -lat

--2022-03-29 09:36:08--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-03-29 09:36:08--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-03-29 09:36:08--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [16]:
vocab,embeddings = [],[]
with open('glove.6B.50d.txt','rt') as fi:
    full_content = fi.read().strip().split('\n')
for i in range(len(full_content)):
    i_word = full_content[i].split(' ')[0]
    i_embeddings = [float(val) for val in full_content[i].split(' ')[1:]]
    vocab.append(i_word)
    embeddings.append(i_embeddings)

In [17]:
vocab_npa = np.array(vocab)
embs_npa = np.array(embeddings)

In [18]:
#insert '<pad>' and '<unk>' tokens at start of vocab_npa.
vocab_npa = np.insert(vocab_npa, 0, '<pad>')
vocab_npa = np.insert(vocab_npa, 1, '<unk>')
print(vocab_npa[:10])

pad_emb_npa = np.zeros((1,embs_npa.shape[1]))   #embedding for '<pad>' token.
unk_emb_npa = np.mean(embs_npa,axis=0,keepdims=True)    #embedding for '<unk>' token.

#insert embeddings for pad and unk tokens at top of embs_npa.
embs_npa = np.vstack((pad_emb_npa,unk_emb_npa,embs_npa))
print(embs_npa.shape)

['<pad>' '<unk>' 'the' ',' '.' 'of' 'to' 'and' 'in' 'a']
(400002, 50)


In [19]:
my_embedding_layer = torch.nn.Embedding.from_pretrained(torch.from_numpy(embs_npa).float())

assert my_embedding_layer.weight.shape == embs_npa.shape
print(my_embedding_layer.weight.shape)

torch.Size([400002, 50])


In [20]:
def convert_text_to_id(df,vocab,unk_token,pad_to_len,pad_token):
  word2idx = {term:idx for idx,term in enumerate(vocab)}
  idx2word = {idx:word for word,idx in word2idx.items()}

  for i in range(len(df)):
    deficit = pad_to_len - len(df[i])
    df[i].extend([pad_token]*deficit)
    for j in range(len(df[i])):
      if df[i][j] not in word2idx:
        df[i][j] = word2idx[unk_token]
      else:
        df[i][j] = word2idx[df[i][j]]
  return df


Xtrain_id = convert_text_to_id(Xtrain,vocab_npa,'<unk>',max_seq_len,'<pad>')


In [21]:
Xtest_id = convert_text_to_id(Xtest,vocab_npa,'<unk>',max_seq_len_test,'<pad>')

In [22]:
len(Xtrain_id[0])

102

In [23]:
Xtrain_id

0       [205, 3, 2017, 3, 199, 61, 224, 12, 9, 308, 13...
1       [83, 348, 14, 16, 21655, 36, 16, 590, 38, 221,...
2       [104, 90, 83, 1704, 190, 22, 45, 277, 97, 6, 1...
3       [90, 83, 590, 271, 102, 190, 43, 90, 38, 4, 22...
4       [43, 5022, 83, 34, 1, 104, 5286, 55, 90, 190, ...
                              ...                        
5085    [1616, 7, 7216, 407, 1169, 24, 78, 4, 0, 0, 0,...
5086    [199, 121, 83, 348, 14, 190, 43, 1060, 1616, 3...
5087    [43, 17, 2862, 6, 2057, 46, 4320, 4, 0, 0, 0, ...
5088    [22, 3073, 1536, 355, 4, 378, 28, 631, 73332, ...
5089    [199, 1235, 837, 16, 4, 0, 0, 0, 0, 0, 0, 0, 0...
Name: utterance, Length: 5090, dtype: object

In [24]:
#Load dataset
Xtrain_id = np.array(Xtrain_id)
Xtrain_id= np.vstack(Xtrain_id).astype('int')
ytrain = np.array(ytrain)
ytrain = ytrain - 1
train_data = TensorDataset(torch.from_numpy(Xtrain_id), torch.from_numpy(ytrain))


In [25]:
Xtest_id = np.array(Xtest_id)
Xtest_id= np.vstack(Xtest_id).astype('int')
ytest = np.array(ytest)
ytest = ytest - 1
test_data = TensorDataset(torch.from_numpy(Xtest_id), torch.from_numpy(ytest))

In [26]:
ytrain

array([2, 3, 1, ..., 0, 0, 0])

In [27]:
trainDataLoader = DataLoader(train_data,batch_size=32)

In [28]:
testDataLoader = DataLoader(test_data, batch_size=32)

In [74]:
class LSTMClass(nn.Module):
    def __init__(self,embs_nps,hidden_size,num_layers,drop_prob):
        super().__init__()
        self.vocab_size = embs_npa.shape[0]
        self.embedding_dim = embs_npa.shape[1]
        self.embedding = nn.Embedding.from_pretrained(torch.from_numpy(embs_npa).float(),freeze=True)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.pool = nn.AdaptiveAvgPool2d((1,self.embedding_dim))
        self.lstm = nn.LSTM(input_size=self.embedding_dim,hidden_size=self.hidden_size,num_layers=self.num_layers,batch_first=True,bidirectional=False)
        self.dropout = nn.Dropout(drop_prob)
        self.linear = nn.Linear(hidden_size, 4)
        # self.softmax = nn.Softmax()
        

    def forward(self, x):
      embed_out = self.embedding(x)
      # print("emb",embed_out.shape)
      sentence_embed_out = self.pool(embed_out)
      # print("Senemb",sentence_embed_out.shape)
      hnot = torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(device)
      cnot = torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(device)
      out,_ = self.lstm(sentence_embed_out,(hnot,cnot))
      out = out[:,-1,:]
      # print("lstm",out.shape)
      # out = self.dropout(out)
      out = self.linear(out)
      # print("lin",out.shape)
      # out = self.softmax(out)
      return out

In [75]:
lstm = LSTMClass(embs_npa,256,1,0.2).to(device)
criterion = nn.CrossEntropyLoss()
# criterion = nn.L1Loss()
optimizer = optim.Adam(lstm.parameters(), lr=0.1)

In [79]:
totalSteps = len(trainDataLoader)
numEpochs = 10
X = 2  # X previous utterances for t th utterance
for epoch in range(numEpochs):  # loop over the dataset multiple times

    running_loss = 0.0
    
    for i, data in tqdm(enumerate(trainDataLoader)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        # print("l",labels.shape)
        # print(labels)
        labels = labels.reshape((-1,)).to(device)
        # print("l",labels.shape)
        # print(labels)
        inputs = torch.squeeze(inputs,0)
        # print(inputs.shape)
        # print(inputs)
        
        outputs = lstm(inputs)
        
          
        # outputs = torch.squeeze(outputs)
        # outputs = outputs.reshape((-1, 1))
        # print(outputs.shape)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
        if (i+1) % 10 == 0:
            print(f'Epoch {epoch + 1} / {numEpochs}, Step {i+1} / {totalSteps}, Loss: {loss.item():.4f}')
            # print(outputs)

print('Finished Training')

11it [00:00, 104.58it/s]

Epoch 1 / 10, Step 10 / 5086, Loss: 1.4224
Epoch 1 / 10, Step 20 / 5086, Loss: 0.0065


44it [00:00, 90.20it/s]

Epoch 1 / 10, Step 30 / 5086, Loss: 3.3861
Epoch 1 / 10, Step 40 / 5086, Loss: 0.5834
Epoch 1 / 10, Step 50 / 5086, Loss: 0.5184


81it [00:00, 107.99it/s]

...

Epoch 10 / 10, Step 5000 / 5086, Loss: 1.9314
Epoch 10 / 10, Step 5010 / 5086, Loss: 0.5523
Epoch 10 / 10, Step 5020 / 5086, Loss: 3.4268


5057it [00:36, 136.22it/s]

Epoch 10 / 10, Step 5030 / 5086, Loss: 0.0134
Epoch 10 / 10, Step 5040 / 5086, Loss: 2.9024
Epoch 10 / 10, Step 5050 / 5086, Loss: 1.4293


5086it [00:36, 139.75it/s]

Epoch 10 / 10, Step 5060 / 5086, Loss: 5.4241
Epoch 10 / 10, Step 5070 / 5086, Loss: 2.8979
Epoch 10 / 10, Step 5080 / 5086, Loss: 0.5797
Finished Training


In [80]:
with torch.no_grad():
    nCorrect = 0
    nSamples = 0
    nClassCorrect = [0 for i in range(4)]
    nClassTotal = [0 for i in range(4)]
    
        
    for i,data in tqdm(enumerate(testDataLoader)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        # print("l",labels.shape)
        # print(labels)
        labels = labels.reshape((-1,)).to(device)
        # print("l",labels.shape)
        # print(labels)
    
        outputs = lstm(inputs)
        # outputs = torch.squeeze(outputs)
        # outputs = outputs.reshape((-1, 1))
        # print(outputs.shape)
        _, predictions = torch.max(outputs,1)
        print(predictions)
        print("l",labels)
        nSamples += labels.shape[0]
        # nCorrect += (predictions == labels).sum().item()
        for i in range(len(predictions)):
            if predictions[i] == labels[i]:
                nCorrect += 1

    
    acc = 100.0 * nCorrect / nSamples
    print(f'Accuracy on test: {acc:.2f}%')

    

23it [00:00, 231.98it/s]

tensor([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 0, 0, 0])
l tensor([2, 3, 2, 1, 0, 2, 3, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 0, 0, 0])
tensor([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 1, 0, 1, 0, 0, 0, 0])
l tensor([0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 2, 3, 2, 3, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 1, 1, 1, 0, 1, 0, 2])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
        0, 1, 0, 0, 0, 1, 0, 1])
l tensor([1, 0, 3, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
        0, 1, 0, 0, 0, 1, 0, 1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1])
l tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1])
tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
        

In [33]:
outputs.shape

torch.Size([18, 4])

In [34]:
labels.shape

torch.Size([18])

In [35]:
ytrain.shape

(5090,)

Dataloader for taking only previous x utterances (Sliding window --> if x utterances to be considered send x+1 samples at a time (x+1 is window size)) 

In [64]:
class MyDataset(Dataset):
    def __init__(self, data, window):
        self.data = data
        self.window = window

    def __getitem__(self, index):
        x = self.data[index:index+self.window]
        return x

    def __len__(self):
        return len(self.data) - self.window

In [65]:
trainDataLoader = DataLoader(MyDataset(train_data,4),batch_size=1)

In [47]:
MyDataset(train_data,4)